In [35]:
import numpy as np
from lookup_table import CaseNum2EdgeOffset, getCaseNum
import trimesh
import os
import time

In [36]:
def marching_cube(thres,cells):
    # vertices use dictionary to avoid duplicate axes
    vertex_array = {}
    face_array = []
    t1 = time.time()
    # -------------------TODO------------------ 
    # compute vertices and faces
    # vertices: [N, 3]
    # faces: [M, 3], e.g. np.array([[0,1,2]]) means a triangle composed of vertices[0], vertices[1] and vertices[2]
    # for-loop is allowed to reduce difficulty
    # -------------------TODO------------------ 
    for x in range(cells.shape[0]-1):
        for y in range(cells.shape[1]-1):
            for z in range(cells.shape[2]-1):
                case_nums = getCaseNum(x, y, z, thres, cells)
                tmp = []
                for case_num in case_nums:
                    if case_num == -1:
                        continue
                    corner0 = (x+CaseNum2EdgeOffset[case_num][0],y+CaseNum2EdgeOffset[case_num][1],z+CaseNum2EdgeOffset[case_num][2])
                    corner1 = (x+CaseNum2EdgeOffset[case_num][3],y+CaseNum2EdgeOffset[case_num][4],z+CaseNum2EdgeOffset[case_num][5])
                    xp,yp,zp = (-cells[corner0])/(cells[corner1]-cells[corner0]) * (np.array(corner1) - np.array(corner0)) + np.array(corner0)
                    if vertex_array.get((xp, yp, zp)) == None:
                        vertex_array[(xp, yp, zp)] = len(vertex_array)
                    tmp.append(vertex_array[(xp, yp, zp)])
                    if len(tmp) == 3:
                        face_array.append(tmp)
                        tmp = []
    l = np.zeros((len(vertex_array), 3))
    for key, value in vertex_array.items():
        l[value, :] = key
    vertex_array = l
    t2 = time.time()
    print("\nTime taken by algorithm\n"+'-'*40+"\n{} s".format(t2-t1))
    
    return np.array(vertex_array), np.array(face_array)

In [37]:
# reconstruct these two animals
shape_name_lst = ['spot', 'bob']
for shape_name in shape_name_lst:
    data = np.load(os.path.join('data', shape_name + '_cell.npy'))
    verts, faces = marching_cube(0, data)
    mesh = trimesh.Trimesh(vertices=verts, faces=faces)
    mesh_txt = trimesh.exchange.obj.export_obj(mesh)
    with open(os.path.join('../results', shape_name + '.obj'),"w") as fp:
        fp.write(mesh_txt)


Time taken by algorithm
----------------------------------------
4.193957090377808 s

Time taken by algorithm
----------------------------------------
4.3914289474487305 s
